Đọc dữ liệu

In [3]:
import pandas as pd
df = pd.read_csv('framingham.csv')
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [4]:
df = df.drop(columns=['education'])

In [5]:
df.isnull().sum()

male                 0
age                  0
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [6]:
df.shape

(4240, 15)

Tiền xử lí dữ liệu

Điền giá trị còn thiếu

In [7]:
# Định nghĩa cột nhị phân
bin_cols = ["male", "currentSmoker", "prevalentStroke", "prevalentHyp", "diabetes"]

# Điền giá trị thiếu cho các đặc trưng nhị phân bằng giá trị thường gặp nhất (mode)
for col in bin_cols:
    mode_val = df[col].mode()[0]
    df[col] = df[col].fillna(mode_val)

missing_values = df.isnull().sum()

In [8]:
import numpy as np

# Điền giá trị thiếu cho các đặc trưng số
numeric_cols = ["cigsPerDay", "BPMeds", "totChol", "BMI", "heartRate", "glucose"]
for col in numeric_cols:
    median_val = df[col].median()
    df[col].fillna(median_val)

missing_values = df.isnull().sum()

Cân bằng dữ liệu

In [9]:
df['TenYearCHD'].value_counts()

TenYearCHD
0    3596
1     644
Name: count, dtype: int64

In [10]:
from sklearn.utils import resample

# Tách lớp đa số và lớp thiểu số
df_majority = df[df['TenYearCHD'] == 0]
df_minority = df[df['TenYearCHD'] == 1]

# Tăng cường lớp thiểu số
df_minority_upsampled = resample(df_minority,
                                 replace=True,    
                                 n_samples=len(df_majority),    
                                 random_state=42) 

# Kết hợp lớp đa số với lớp thiểu số đã tăng cường
df_balanced = pd.concat([df_majority, df_minority_upsampled])

In [11]:
df_balanced['TenYearCHD'].value_counts()

TenYearCHD
0    3596
1    3596
Name: count, dtype: int64

Train Test với dữ liệu gốc 

Train Test Split and Feature Scalling

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Tách các đặc trưng (X) và biến mục tiêu (y)
X = df_balanced.drop(columns=['TenYearCHD'])
y = df_balanced['TenYearCHD']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tiền xử lí dữ liệu
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

Scaling (StandardScale)

In [13]:
# Khởi tạo StandardScaler
scaler = StandardScaler()

# Định hình scaler cho dữ liệu huấn luyện và biến đổi cả dữ liệu huấn luyện và dữ liệu kiểm tra
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

Training 2 Models With Different Metrics

In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

# Định nghĩa danh sách các bộ phân loại
classifiers = [
    KNeighborsClassifier(),
    GaussianNB(),
    BernoulliNB()
]

# Tạo một từ điển để lưu trữ kết quả
results = {}

# Huấn luyện và đánh giá từng bộ phân loại
for clf in classifiers:
    clf_name = clf.__class__.__name__
    clf.fit(X_train_scaled, y_train)   
    y_pred = clf.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"{clf_name} Accuracy: {accuracy}")

    print(f"Classification Report for {clf_name}:")
    print(classification_report(y_test, y_pred))

    print(f"Confusion Matrix for {clf_name}:")
    print(confusion_matrix(y_test, y_pred))
    print("="*50)

KNeighborsClassifier Accuracy: 0.7908269631688672
Classification Report for KNeighborsClassifier:
              precision    recall  f1-score   support

           0       0.90      0.66      0.76       735
           1       0.72      0.92      0.81       704

    accuracy                           0.79      1439
   macro avg       0.81      0.79      0.79      1439
weighted avg       0.81      0.79      0.79      1439

Confusion Matrix for KNeighborsClassifier:
[[487 248]
 [ 53 651]]
GaussianNB Accuracy: 0.5816539263377345
Classification Report for GaussianNB:
              precision    recall  f1-score   support

           0       0.56      0.91      0.69       735
           1       0.71      0.24      0.36       704

    accuracy                           0.58      1439
   macro avg       0.63      0.57      0.52      1439
weighted avg       0.63      0.58      0.53      1439

Confusion Matrix for GaussianNB:
[[667  68]
 [534 170]]
BernoulliNB Accuracy: 0.6337734537873523
Classif

Show Each Model Results

In [16]:
# Định nghĩa một danh sách các bộ phân loại
classifiers = [
    KNeighborsClassifier(),
    GaussianNB(),
    BernoulliNB()
]

# Tạo một DataFrame để lưu trữ kết quả
results = []

# Huấn luyện và đánh giá từng bộ phân loại
for clf in classifiers:
    clf_name = clf.__class__.__name__  
    clf.fit(X_train_scaled, y_train)  
    y_pred = clf.predict(X_test_scaled)  

    accuracy = accuracy_score(y_test, y_pred)  
    report = classification_report(y_test, y_pred, output_dict=True)  
    f1_score = report['weighted avg']['f1-score']  
    precision = report['weighted avg']['precision']  
    recall = report['weighted avg']['recall']  

    # Thêm kết quả vào DataFrame
    results.append({'Model': clf_name, 'Accuracy': accuracy, 'F1-Score': f1_score,
                    'Precision': precision, 'Recall': recall})

results_df = pd.DataFrame(results)  
results_df  

,Model,Accuracy,F1-Score,Precision,Recall
0,KNeighborsClassifier,0.790827,0.787554,0.814909,0.790827
1,GaussianNB,0.581654,0.528526,0.633116,0.581654
2,BernoulliNB,0.633773,0.632547,0.634254,0.633773


 Best Model

In [17]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

# Khởi tạo 
rf_classifier = KNeighborsClassifier()

# Huấn luyện 
rf_classifier.fit(X_train_scaled, y_train)

# Dự đoán trên tập kiểm tra
y_pred_rf = rf_classifier.predict(X_test_scaled)

# Tính toán độ chính xác
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Độ chính xác của KNeighborsClassifier:", accuracy_rf)

# Báo cáo phân loại
print("Báo cáo phân loại cho KNeighborsClassifier:")
print(classification_report(y_test, y_pred_rf))

# Ma trận nhầm lẫn
print("Ma trận nhầm lẫn cho KNeighborsClassifier:")
print(confusion_matrix(y_test, y_pred_rf))

Độ chính xác của KNeighborsClassifier: 0.7908269631688672
Báo cáo phân loại cho KNeighborsClassifier:
              precision    recall  f1-score   support

           0       0.90      0.66      0.76       735
           1       0.72      0.92      0.81       704

    accuracy                           0.79      1439
   macro avg       0.81      0.79      0.79      1439
weighted avg       0.81      0.79      0.79      1439

Ma trận nhầm lẫn cho KNeighborsClassifier:
[[487 248]
 [ 53 651]]


Single prediction

In [18]:
# test 1:
print("Test1:")
print("predcted class ",rf_classifier.predict(X_test_scaled[10].reshape(1,-1))[0])
print("actual class ", y_test.iloc[10])

# test 2:
print("Test2:")
print("predcted class ",rf_classifier.predict(X_test_scaled[200].reshape(1,-1))[0])
print("actual class ", y_test.iloc[200])

# test 3:
print("Test3:")
print("predcted class ",rf_classifier.predict(X_test_scaled[110].reshape(1,-1))[0])
print("actual class ", y_test.iloc[110])

Test1:
predcted class  0
actual class  0
Test2:
predcted class  1
actual class  1
Test3:
predcted class  1
actual class  1


Save Model

In [19]:
import pickle
pickle.dump(rf_classifier,open("rf_classifier.pkl",'wb'))
pickle.dump(scaler,open("scaler.pkl",'wb'))

Load model to test 1

In [20]:
import pickle

# Tải mô hình 
with open("rf_classifier.pkl", "rb") as file:
    rf_classifier = pickle.load(file)

# Tải bộ chuẩn hóa
with open("scaler.pkl", "rb") as file:
    scaler = pickle.load(file)

Predictive System

In [21]:
def predict(model, scaler, male, age, currentSmoker, cigsPerDay, BPMeds, prevalentStroke, prevalentHyp, diabetes, totChol, sysBP, diaBP, BMI, heartRate, glucose):
    # Mã hóa các biến phân loại
    male_encoded = 1 if male.lower() == "male" else 0
    currentSmoker_encoded = 1 if currentSmoker.lower() == "yes" else 0
    BPMeds_encoded = 1 if BPMeds.lower() == "yes" else 0
    prevalentStroke_encoded = 1 if prevalentStroke.lower() == "yes" else 0
    prevalentHyp_encoded = 1 if prevalentHyp.lower() == "yes" else 0
    diabetes_encoded = 1 if diabetes.lower() == "yes" else 0

    # Chuẩn bị mảng đặc trưng
    features = np.array([[male_encoded, age, currentSmoker_encoded, cigsPerDay, BPMeds_encoded, prevalentStroke_encoded, prevalentHyp_encoded, diabetes_encoded, totChol, sysBP, diaBP, BMI, heartRate, glucose]])

    # Chuẩn hóa
    scaled_features = scaler.transform(features)

    # Dự đoán bằng mô hình
    result = model.predict(scaled_features)

    return result[0]

In [22]:
# test 1:
male = "female"
age = 56.00
currentSmoker = "yes"
cigsPerDay = 3.00
BPMeds = "no"
prevalentStroke = "no"
prevalentHyp = "yes"
diabetes = 'no'
totChol = 285.00
sysBP = 145.00
diaBP = 100.00
BMI = 30.14
heartRate = 80.00
glucose = 86.00

result = predict(rf_classifier, scaler, male, age, currentSmoker, cigsPerDay, BPMeds, prevalentStroke, prevalentHyp, diabetes, totChol, sysBP, diaBP, BMI, heartRate, glucose)

if result == 1:
    print("The Patient has Heart Diseas")
else:
    print("The Patiennt has No Heart Deseas")

The Patiennt has No Heart Deseas


In [23]:
#Test 2
male = 'female'
age = 63.0
currentSmoker = 'yes'
cigsPerDay = 3.0
BPMeds = 'no'
prevalentStroke = 'no'
prevalentHyp = 'yes'
diabetes = 'no'
totChol = 267.0
sysBP = 156.5
diaBP = 92.5
BMI = 27.1
heartRate = 60.0
glucose = 79.0
result = 1.0

result = predict(rf_classifier, scaler, male, age, currentSmoker, cigsPerDay, BPMeds, prevalentStroke, prevalentHyp, diabetes, totChol, sysBP, diaBP, BMI, heartRate, glucose)

if result == 1:
    print("The Patient has Heart Diseas")
else:
    print("The Patiennt has No Heart Deseas")

The Patient has Heart Diseas
